In [1]:
import pulp
import pandas as pd

In [2]:
def instancia_segundaVA_individual():
    turmas = ["1A", "2A", "3A"]
    ano_turma = {"1A": 1, "2A": 2, "3A": 3}

    disciplinas = ["Matematica", "Fisica", "Quimica"]

    professores = ["prof1", "prof2", "prof3"]

    horarios = ["Seg_M", "Ter_M", "Qua_M", "Qui_M", "Sex_M"]

    quantidade_salas = 3

    professor_pode_lecionar = {
        ("prof1", "Matematica"): 1,
        ("prof2", "Fisica"): 1,
        ("prof3", "Quimica"): 1,
    }

    disponibilidade = {}
    for p in professores:
        for h in horarios:
            disponibilidade[(p, h)] = 1

    quantidade_aulas = {}
    for turma in turmas:
        quantidade_aulas[(turma, "Matematica")] = 1
        quantidade_aulas[(turma, "Fisica")] = 1
        quantidade_aulas[(turma, "Quimica")] = 1

    atribuicoes_fixas = {}
    disciplinas_programas = {}
    disciplinas_programas_externos = {}

    return {
        "turmas": turmas,
        "ano_da_turma": ano_turma,
        "disciplinas": disciplinas,
        "professores": professores,
        "horarios": horarios,
        "quantidade_salas": quantidade_salas,
        "professor_pode_lecionar": professor_pode_lecionar,
        "professor_disponivel_no_horario": disponibilidade,
        "quantidade_aulas_requeridas": quantidade_aulas,
        "atribuicoes_fixas": atribuicoes_fixas,
        "disciplinas_programas": disciplinas_programas,
        "disciplinas_programas_externos": disciplinas_programas_externos,
    }

In [4]:
def construir_e_resolver_modelo(instancia, limite_tempo=120):
    turmas = instancia["turmas"]
    ano_da_turma = instancia["ano_da_turma"]
    disciplinas = instancia["disciplinas"]
    professores = instancia["professores"]
    horarios = instancia["horarios"]
    quantidade_salas = instancia["quantidade_salas"]
    professor_pode_lecionar = instancia["professor_pode_lecionar"]
    professor_disponivel_no_horario = instancia["professor_disponivel_no_horario"]

    quantidade_aulas_requeridas = {
        (t, d): instancia["quantidade_aulas_requeridas"].get((t, d), 0)
        for t in turmas
        for d in disciplinas
    }

    atribuicoes_fixas = set(instancia.get("atribuicoes_fixas", []))
    disciplinas_programas = set(instancia.get("disciplinas_programas", []))
    disciplinas_programas_externos = set(instancia.get("disciplinas_programas_externos", []))

    # Contabiliza aulas fixas já alocadas
    ocupacao_fixa_por_horario = {h: 0 for h in horarios}
    for (turma, disc, prof, horario) in atribuicoes_fixas:
        ocupacao_fixa_por_horario[horario] += 1

    # Ajusta a carga requerida reduzindo o que já está fixo
    for (turma, disc, prof, horario) in atribuicoes_fixas:
        if (turma, disc) in quantidade_aulas_requeridas:
            if quantidade_aulas_requeridas[(turma, disc)] > 0:
                quantidade_aulas_requeridas[(turma, disc)] -= 1

    #  VARIÁVEIS DE DECISÃO
    #  x[turma, disciplina, professor, horario] = 1 se a aula ocorre

    variavel_alocacao = {}

    for turma in turmas:
        for disciplina in disciplinas:
            for professor in professores:
                for horario in horarios:

                    # Ignorar combinações que já estão fixas
                    if (turma, disciplina, professor, horario) in atribuicoes_fixas:
                        continue

                    # Professor não leciona a disciplina
                    if professor_pode_lecionar.get((professor, disciplina), 0) == 0:
                        continue

                    # Professor indisponível no horário
                    if professor_disponivel_no_horario.get((professor, horario), 0) == 0:
                        continue

                    variavel_alocacao[(turma, disciplina, professor, horario)] = (
                        pulp.LpVariable(
                            f"alocar_{turma}_{disciplina}_{professor}_{horario}",
                            cat="Binary"
                        )
                    )

    # Objetivo zero (somente factibilidade)
    modelo = pulp.LpProblem("Alocacao_Aulas_Escolar")
    modelo += 0, "Objetivo_Nulo"

    #  RESTRIÇÃO 1 — COBERTURA DAS AULAS
    for turma in turmas:
        for disciplina in disciplinas:

            soma_variaveis = pulp.lpSum(
                variavel_alocacao.get((turma, disciplina, professor, horario), 0)
                for professor in professores
                for horario in horarios
            )

            aulas_fixas = sum(
                1 for (t, d, _, _) in atribuicoes_fixas
                if t == turma and d == disciplina
            )

            modelo += (
                soma_variaveis + aulas_fixas == quantidade_aulas_requeridas[(turma, disciplina)],
                f"cobertura_{turma}_{disciplina}"
            )

    #  RESTRIÇÃO 2 — UMA AULA POR TURMA POR HORÁRIO
    for turma in turmas:
        for horario in horarios:

            soma_turma_horario = pulp.lpSum(
                variavel_alocacao.get((turma, disciplina, professor, horario), 0)
                for disciplina in disciplinas
                for professor in professores
            )

            aulas_fixas = sum(
                1 for (t, _, _, h) in atribuicoes_fixas
                if t == turma and h == horario
            )

            modelo += (
                soma_turma_horario + aulas_fixas <= 1,
                f"uma_aula_por_turma_no_horario_{turma}_{horario}"
            )

    #  RESTRIÇÃO 3 — UM PROFESSOR POR HORÁRIO
    for professor in professores:
        for horario in horarios:

            soma_professor_horario = pulp.lpSum(
                variavel_alocacao.get((turma, disciplina, professor, horario), 0)
                for turma in turmas
                for disciplina in disciplinas
            )

            aulas_fixas = sum(
                1 for (_, _, p, h) in atribuicoes_fixas
                if p == professor and h == horario
            )

            modelo += (
                soma_professor_horario + aulas_fixas <= 1,
                f"um_professor_por_horario_{professor}_{horario}"
            )

    #  RESTRIÇÃO 4 — LIMITE DE SALAS POR HORÁRIO
    for horario in horarios:

        soma_ocupacao = pulp.lpSum(
            variavel_alocacao[var]
            for var in variavel_alocacao
            if var[3] == horario
        )

        aulas_fixas = ocupacao_fixa_por_horario.get(horario, 0)

        modelo += (
            soma_ocupacao + aulas_fixas <= quantidade_salas,
            f"limite_salas_{horario}"
        )
    
    

    #  SOLVER
    solver = pulp.PULP_CBC_CMD(msg=True, timeLimit=limite_tempo, threads=2)
    status = modelo.solve(solver)

    print("\nSTATUS DO SOLVER:", pulp.LpStatus[modelo.status])

    atribuicoes = []

    # Atribuições fixas
    for (turma, disc, professor, horario) in atribuicoes_fixas:
        atribuicoes.append({
            "turma": turma,
            "disciplina": disc,
            "professor": professor,
            "horario": horario,
            # "fixa": True
        })

    # Atribuições decididas pelo modelo
    for chave, var in variavel_alocacao.items():
        if pulp.value(var) is not None and pulp.value(var) > 0.5:
            turma, disc, professor, horario = chave
            atribuicoes.append({
                "turma": turma,
                "disciplina": disc,
                "professor": professor,
                "horario": horario,
                # "fixa": False
            })

    df = pd.DataFrame(atribuicoes)
    if not df.empty:
        df = df.sort_values(["horario", "turma", "disciplina"]).reset_index(drop=True)

    return modelo, df

In [5]:
instancia = instancia_segundaVA_individual()
modelo, tabela = construir_e_resolver_modelo(instancia)

print("Objetivos:", pulp.value(modelo.objective))
print("Restrições:", len(modelo.constraints))
print("Variáveis:", len(modelo.variables()))
if tabela.empty:
    print("Nenhuma atribuição encontrada — instância possivelmente inviável ou precisa de tempo maior.")
else:
    print("\nGrade encontrada:")
    print(tabela.to_string(index=False))



STATUS DO SOLVER: Optimal
Objetivos: None
Restrições: 44
Variáveis: 46

Grade encontrada:
turma disciplina professor horario
   2A Matematica     prof1   Qua_M
   3A    Quimica     prof3   Qua_M
   1A Matematica     prof1   Qui_M
   3A     Fisica     prof2   Qui_M
   1A     Fisica     prof2   Seg_M
   2A    Quimica     prof3   Sex_M
   3A Matematica     prof1   Sex_M
   1A    Quimica     prof3   Ter_M
   2A     Fisica     prof2   Ter_M
